In [161]:
#Importación modulos

import pandas as pd
import numpy as np
import re
import nltk
from nltk import ngrams
import spacy



In [162]:
#df_glass_completo=pd.read_csv("glassdoor_reviews.csv")

In [163]:
#Se hace un DF mas pequeño para trabajar mas facilemte
#df_glass_mil =df_glass_completo.head(1000)

#df_glass_mil.to_csv('glassdor_mil.csv', index=False)


In [164]:
#Crea df con prueba y test
#train_set = df_glass_mil.sample(frac=0.8, random_state=42) 
#train_set.to_csv('glassdor_train.csv', index=False)

# Drop de los datos que estan en el set train
 
#test_set = df_glass_mil.drop(train_set.index) 
#test_set.to_csv('glassdor_test.csv', index=False)


In [165]:
#Se trabaja con el train df

df_glass=pd.read_csv("glassdor_train.csv")
df_glass.head(3)

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons
0,ALDI,2019-07-05,Deputy Store Manager,"Current Employee, more than 5 years","Chatteris, England, England",5,1.0,2.0,NaN,2.0,1.0,1.0,v,v,v,I do not know what to say,Every day is different 😉,"No social life after work, because I'm over t..."
1,ALDI,2020-09-01,Store Assistant Manager,"Current Employee, more than 5 years","Neston, North West England, England, England",5,4.0,4.0,NaN,4.0,4.0,4.0,v,r,v,good,I like it here a lot,long hours and don't like customers
2,ALDI,2020-09-03,Shop Assistant,"Current Employee, more than 3 years","Huddersfield, England, England",5,5.0,4.0,NaN,3.0,3.0,4.0,o,o,o,Hard work,Good pay rate \r\nFlexible hours\r\nNice colle...,Hard work \r\nHard work \r\nDisrespectful cust...


In [166]:
#Replace NaN with , Na values interfere in the merge and re code

df_glass['headline'] = df_glass['headline'].fillna(",")
df_glass['pros'] = df_glass['pros'].fillna(",")
df_glass['cons'] = df_glass['cons'].fillna(",")

In [167]:
#Merge hedline, pros, cons

df_glass["headline+pros+cons"] = df_glass["headline"] + " " + df_glass["pros"]+ " " + df_glass["cons"]

df_glass["headline+pros+cons"]

0      I do not know what to say Every day is differe...
1      good I like it here a lot long hours and don't...
2      Hard work Good pay rate \r\nFlexible hours\r\n...
3      The pay attracts everyone...dont be fooled. Fe...
4      If you don't brown nose you won't progress Gre...
                             ...                        
795    Hard work Good payment, nice team, always diff...
796    Good staff!! amazing managers, good staff, gre...
797    Not bad Good pay ... organise company alot of ...
798    mixed Well paid. Flexible working patterns at ...
799    store assistant review good pay, fair manageme...
Name: headline+pros+cons, Length: 800, dtype: object

In [168]:
#Check if there is an Na in "headline+pros+cons"

df_glass[df_glass["headline+pros+cons"].isnull()]

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons,headline+pros+cons


In [169]:
#Make the info at column "headline+pros+cons" string

df_glass['headline+pros+cons']= df_glass['headline+pros+cons'].astype('string')

# Text Preprocessing

In [170]:
#Remove Punctuation Marks

def quitar_puntuacion(texto):
    return re.sub(r'[^\w\s]',"", texto)

# Apply function to the merged column

df_glass['headline+pros+cons'] = df_glass['headline+pros+cons'].apply(quitar_puntuacion)

#Remove Stop words
def quitar_stop(texto):
    return re.sub(r'\b(?:and|an|at|a|of|on|I|for|with|the|at|from|in|to)\b', "", texto, flags=re.IGNORECASE)

# Apply function to the merged column

df_glass['headline+pros+cons'] = df_glass['headline+pros+cons'].apply(quitar_stop)



In [171]:
#The info ended as type object, have to convert it to string
df_glass['headline+pros+cons']

0       do not know what  say Every day is different ...
1      good  like it here  lot long hours  dont like ...
2      Hard work Good pay rate \r\nFlexible hours\r\n...
3       pay attracts everyonedont be fooled Fellow co...
4      If you dont brown nose you wont progress Great...
                             ...                        
795    Hard work Good payment nice team always differ...
796    Good staff amazing managers good staff great e...
797    Not bad Good pay  organise company alot  overt...
798    mixed Well paid Flexible working patterns  sto...
799    store assistant review good pay fair managemen...
Name: headline+pros+cons, Length: 800, dtype: object

In [172]:
#Lematize
nlp = spacy.load("en_core_web_sm")

def lematizar_(text):
    obj = nlp(text.lower())
    lemmatize_token = [x.lemma_ for x in obj]
    return lemmatize_token

# Aplicamos la lematización a cada fila
df_glass['headline+pros+cons_lemma'] = df_glass['headline+pros+cons'].apply(lematizar_)

In [173]:
#Hacer lemma string
df_glass['headline+pros+cons_lemma']=df_glass['headline+pros+cons_lemma'].astype('string')

In [174]:
#Remove Symbols Stop words
def quitar_stop(texto):
    return re.sub(r"[ \[\],']", " ", texto, flags=re.IGNORECASE)

# Apply function to the merged column

df_glass['headline+pros+cons_cleanlemma'] = df_glass['headline+pros+cons_lemma'].apply(quitar_stop)

In [175]:
#Create new DF with bigrams to record Y values (recommend)

df_bigrams_recommend=pd.DataFrame({"count":[],"Recommend":[]})

cc = 0

for x in df_glass['headline+pros+cons_cleanlemma']:

    text = df_glass['headline+pros+cons_cleanlemma'].iloc[cc]
    #recommend = df_glass['recommend'].iloc[cc]

    tokens = nltk.word_tokenize(text)

    unigrams=list(ngrams(tokens, 1))
    bigrams = list(ngrams(tokens, 2))
    trigrams = list(ngrams(tokens, 3))

    Ngrams=-1 #To record all

    #unigram_counts = (pd.Series(unigrams).value_counts())[:Ngrams]

    bigram_counts = (pd.Series(bigrams).value_counts())[:Ngrams]
    
    #trigram_counts = (pd.Series(trigrams).value_counts())[:Ngrams]
    

    df_new = bigram_counts.to_frame()

    df_new["Recommend"]= df_glass['recommend'].iloc[cc]

    df_bigrams_recommend = pd.concat([df_new, df_bigrams_recommend])
    cc = cc+ 1


In [176]:
bigram_counts

(store, assistant)     1
(assistant, review)    1
(review, good)         1
(good, pay)            1
(pay, fair)            1
(fair, management)     1
(management, good)     1
(good, social)         1
(social, life)         1
(life, hard)           1
(hard, work)           1
(work, long)           1
(long, hour)           1
(hour, short)          1
Name: count, dtype: int64

In [177]:
#Index a columna
df_bigrams_recommend=df_bigrams_recommend.reset_index()

In [178]:
#Save the DF with bigrams on a CSV
df_bigrams_recommend.to_csv('Bigrams.csv', index=False)

In [179]:
#Se quitan los duplicados del reporte de recommend, y se remueve la columna count

valores_bigrams_unicos = df_bigrams_recommend.drop_duplicates(subset=['index','Recommend'])
valores_bigrams_unicos = valores_bigrams_unicos.drop('count',axis=1)


In [180]:
valores_bigrams_unicos

,index,Recommend
0,"(store, assistant)",v
1,"(assistant, review)",v
2,"(review, good)",v
3,"(good, pay)",v
4,"(pay, fair)",v
...,...,...
32212,"(work, because)",v
32216,"(no, social)",v
32217,"(different, no)",v
32221,"(say, every)",v


In [181]:
#Convertimos los valores de la tupla a string, y depues concatenamos

valores_bigrams_unicos["index"]=valores_bigrams_unicos["index"].astype(str)
valores_bigrams_unicos["index+Recommend"]=valores_bigrams_unicos["index"]+ valores_bigrams_unicos["Recommend"]

In [182]:
valores_bigrams_unicos

,index,Recommend,index+Recommend
0,"('store', 'assistant')",v,"('store', 'assistant')v"
1,"('assistant', 'review')",v,"('assistant', 'review')v"
2,"('review', 'good')",v,"('review', 'good')v"
3,"('good', 'pay')",v,"('good', 'pay')v"
4,"('pay', 'fair')",v,"('pay', 'fair')v"
...,...,...,...
32212,"('work', 'because')",v,"('work', 'because')v"
32216,"('no', 'social')",v,"('no', 'social')v"
32217,"('different', 'no')",v,"('different', 'no')v"
32221,"('say', 'every')",v,"('say', 'every')v"


In [183]:
#Agarramos el df de las recomendaciones y hacemos el concatenado anterior

df_bigrams_recommend_modificado=df_bigrams_recommend

df_bigrams_recommend_modificado["index"]=df_bigrams_recommend_modificado["index"].astype(str)
df_bigrams_recommend_modificado["index+Recommend"]=df_bigrams_recommend_modificado["index"]+ df_bigrams_recommend_modificado["Recommend"]

In [184]:
df_bigrams_recommend_modificado

,index,count,Recommend,index+Recommend
0,"('store', 'assistant')",1.0,v,"('store', 'assistant')v"
1,"('assistant', 'review')",1.0,v,"('assistant', 'review')v"
2,"('review', 'good')",1.0,v,"('review', 'good')v"
3,"('good', 'pay')",1.0,v,"('good', 'pay')v"
4,"('pay', 'fair')",1.0,v,"('pay', 'fair')v"
...,...,...,...,...
32219,"('day', 'be')",1.0,v,"('day', 'be')v"
32220,"('every', 'day')",1.0,v,"('every', 'day')v"
32221,"('say', 'every')",1.0,v,"('say', 'every')v"
32222,"('what', 'say')",1.0,v,"('what', 'say')v"


In [186]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Bigrama + Reseña"

valores_bigrams_unicos["Frecuencia Bigrama + Reseña"]=0

colcount= df_bigrams_recommend_modificado.columns.get_loc('count')
colBR=  valores_bigrams_unicos.columns.get_loc("Frecuencia Bigrama + Reseña")


loc=0

for gramm in valores_bigrams_unicos['index+Recommend']:
    cc=0
    loc2=0
    
    for gramm_2 in df_bigrams_recommend_modificado ['index+Recommend']:
        if gramm == gramm_2:
            cc= cc+ df_bigrams_recommend_modificado.iloc[loc2,colcount]
        loc2=loc2+1
    
    valores_bigrams_unicos.iloc[loc,colBR]=cc
    loc=loc+1

In [187]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Bigrama"
valores_bigrams_unicos["Frecuencia Bigrama"]=0

colcount= df_bigrams_recommend_modificado.columns.get_loc('count')
fB= valores_bigrams_unicos.columns.get_loc("Frecuencia Bigrama")

loc=0

for gramm in valores_bigrams_unicos['index']:
    cc=0
    loc2=0
    
    for gramm_2 in df_bigrams_recommend_modificado ['index']:
        if gramm == gramm_2:
            cc= cc+ df_bigrams_recommend_modificado.iloc[loc2,colcount]
        loc2=loc2+1
    
    valores_bigrams_unicos.iloc[loc,fB]=cc
    loc=loc+1


In [188]:
#Hacemos un ciclo For para calcular probabilidad

valores_bigrams_unicos["Probabilidad"]= None
loc=0
colprob= valores_bigrams_unicos.columns.get_loc('Probabilidad')
colFBR= valores_bigrams_unicos.columns.get_loc("Frecuencia Bigrama + Reseña")
colFB= valores_bigrams_unicos.columns.get_loc('Frecuencia Bigrama')

for fecuencia in valores_bigrams_unicos['index']:

    valores_bigrams_unicos.iloc[loc,colprob]=valores_bigrams_unicos.iloc[loc,colFBR]/valores_bigrams_unicos.iloc[loc,colFB]
    loc=loc+1

In [189]:
valores_bigrams_unicos

valores_bigrams_unicos.to_csv('ProbabilidadBiGramm.csv', index=False)

In [190]:
#Hacer tabla prob limpia
df_bigrams_test=pd.DataFrame({"Word":[],"V":[],"X":[],"O":[]})
df_bigrams_test['Word']=valores_bigrams_unicos['index'].drop_duplicates()
df_bigrams_test = df_bigrams_test.fillna(0)

In [191]:
colP=valores_bigrams_unicos.columns.get_loc("Probabilidad")
colRec=valores_bigrams_unicos.columns.get_loc("Recommend")
colV=df_bigrams_test.columns.get_loc("V")
colX=df_bigrams_test.columns.get_loc("X")
colO=df_bigrams_test.columns.get_loc("O")



locw=0
for word in df_bigrams_test['Word']:
    loci=0
    for index in valores_bigrams_unicos['index']:
        if word == index:
            if valores_bigrams_unicos.iloc[loci,colRec]=='v':
                df_bigrams_test.iloc[locw,colV]=valores_bigrams_unicos.iloc[loci,colP]
            if valores_bigrams_unicos.iloc[loci,colRec]=='x':
                df_bigrams_test.iloc[locw,colX]=valores_bigrams_unicos.iloc[loci,colP]
            if valores_bigrams_unicos.iloc[loci,colRec]=='o':
                df_bigrams_test.iloc[locw,colO]=valores_bigrams_unicos.iloc[loci,colP]
        loci=loci+1
    locw=locw+1 


In [192]:
df_bigrams_test.head(4)

,Word,V,X,O
0,"('store', 'assistant')",0.390244,0.439024,0.170732
1,"('assistant', 'review')",1.000000,0.000000,0.000000
2,"('review', 'good')",0.750000,0.250000,0.000000
3,"('good', 'pay')",0.436090,0.353383,0.210526


In [193]:
df_bigrams_test.to_csv('ProbabilidadBiGramm_VXO.csv', index=False)


Hacer lo mismo de Bigrams a Unigrams

In [255]:
#Create new DF with unigrams to record Y values (recommend)

colhpccl= df_glass.columns.get_loc('headline+pros+cons_cleanlemma')
colrm= df_glass.columns.get_loc('recommend')


df_uigrams_recommend=pd.DataFrame({"count":[],"Recommend":[]})

cc = 0

for x in df_glass['headline+pros+cons_cleanlemma']:

    text = df_glass.iloc[cc,colhpccl]
    #recommend = df_glass['recommend'].iloc[cc]

    tokens = nltk.word_tokenize(text)

    unigrams=list(ngrams(tokens, 1))
    #bigrams = list(ngrams(tokens, 2))
    #trigrams = list(ngrams(tokens, 3))

    Ngrams=-1 #To record all

    unigram_counts = (pd.Series(unigrams).value_counts())[:Ngrams]

    #bigram_counts = (pd.Series(bigrams).value_counts())[:Ngrams]
    
    #trigram_counts = (pd.Series(trigrams).value_counts())[:Ngrams]
    

    df_new = unigram_counts.to_frame()

    df_new["Recommend"]= df_glass.iloc[cc,colrm]

    df_uigrams_recommend = pd.concat([df_new, df_uigrams_recommend])
    cc = cc+ 1

In [256]:
df_uigrams_recommend

,count,Recommend
"(good,)",2.0,v
"(store,)",1.0,v
"(assistant,)",1.0,v
"(review,)",1.0,v
"(pay,)",1.0,v
...,...,...
"(day,)",1.0,v
"(every,)",1.0,v
"(say,)",1.0,v
"(what,)",1.0,v


In [196]:
#Index a columna
df_uigrams_recommend = df_uigrams_recommend.reset_index()

In [197]:
#Save the DF with unigrams on a CSV
df_uigrams_recommend.to_csv('Unigrams.csv', index=False)

In [198]:
#Se quitan los duplicados del reporte de recommend, y se remueve la columna count

valores_unigrams_unicos = df_uigrams_recommend.drop_duplicates(subset=['index','Recommend'])
valores_unigrams_unicos = valores_unigrams_unicos.drop('count',axis=1)

In [199]:
valores_unigrams_unicos

,index,Recommend
0,"(good,)",v
1,"(store,)",v
2,"(assistant,)",v
3,"(review,)",v
4,"(pay,)",v
...,...,...
25472,"(pandemic,)",x
25483,"(everyonedont,)",x
25489,"(fool,)",x
25510,"(disrespectful,)",o


In [200]:
#Convertimos los valores de la tupla a string, y depues concatenamos

valores_unigrams_unicos["index"]=valores_unigrams_unicos["index"].astype(str)
valores_unigrams_unicos["index+Recommend"]=valores_unigrams_unicos["index"]+ valores_unigrams_unicos["Recommend"]

In [201]:
#Agarramos el df de las recomendaciones y hacemos el concatenado anterior

df_uigrams_recommend_modificado=df_uigrams_recommend

df_uigrams_recommend_modificado["index"]=df_uigrams_recommend_modificado["index"].astype(str)
df_uigrams_recommend_modificado["index+Recommend"]=df_uigrams_recommend_modificado["index"]+ df_uigrams_recommend_modificado["Recommend"]

In [202]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Unigrama + Reseña"

valores_unigrams_unicos["Frecuencia Unigrama + Reseña"]= 0

colC=df_uigrams_recommend_modificado.columns.get_loc('count')

colFUR=valores_unigrams_unicos.columns.get_loc("Frecuencia Unigrama + Reseña")

loc=0

for gramm in valores_unigrams_unicos['index+Recommend']:
    cc=0
    loc2=0
    
    for gramm_2 in df_uigrams_recommend_modificado ['index+Recommend']:
        if gramm == gramm_2:
            cc = cc + df_uigrams_recommend_modificado.iloc[loc2,colC]
        loc2=loc2+1
    
    valores_unigrams_unicos.iloc[loc,colFUR]=cc
    loc=loc+1

In [203]:
valores_unigrams_unicos

,index,Recommend,index+Recommend,Frecuencia Unigrama + Reseña
0,"('good',)",v,"('good',)v",330
1,"('store',)",v,"('store',)v",70
2,"('assistant',)",v,"('assistant',)v",30
3,"('review',)",v,"('review',)v",10
4,"('pay',)",v,"('pay',)v",222
...,...,...,...,...
25472,"('pandemic',)",x,"('pandemic',)x",1
25483,"('everyonedont',)",x,"('everyonedont',)x",1
25489,"('fool',)",x,"('fool',)x",1
25510,"('disrespectful',)",o,"('disrespectful',)o",1


In [204]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Unigrama"

valores_unigrams_unicos["Frecuencia Unigrama"]=0
loc=0

colC=df_uigrams_recommend_modificado.columns.get_loc('count')
colFu=valores_unigrams_unicos.columns.get_loc('Frecuencia Unigrama')


for gramm in valores_unigrams_unicos['index']:
    cc=0
    loc2=0
    
    for gramm_2 in df_uigrams_recommend_modificado ['index']:
        if gramm == gramm_2:
            cc= cc+ df_uigrams_recommend_modificado.iloc[loc2,colC]
        loc2=loc2+1
    
    valores_unigrams_unicos.iloc[loc,colFu]=cc
    loc=loc+1

In [205]:
#Hacemos un ciclo For para calcular probabilidad

valores_unigrams_unicos["Probabilidad"]=None

colP=valores_unigrams_unicos.columns.get_loc("Probabilidad")
colFUR=valores_unigrams_unicos.columns.get_loc("Frecuencia Unigrama + Reseña")
colFU=valores_unigrams_unicos.columns.get_loc("Frecuencia Unigrama")

loc=0

for fecuencia in valores_unigrams_unicos['index']:

    valores_unigrams_unicos.iloc[loc,colP]=valores_unigrams_unicos.iloc[loc,colFUR]/valores_unigrams_unicos.iloc[loc,colFU]
    loc=loc+1

In [206]:
valores_unigrams_unicos

valores_unigrams_unicos.to_csv('ProbabilidadUniGramm.csv', index=False)


In [207]:
valores_unigrams_unicos

,index,Recommend,index+Recommend,Frecuencia Unigrama + Reseña,Frecuencia Unigrama,Probabilidad
0,"('good',)",v,"('good',)v",330,666,0.495495
1,"('store',)",v,"('store',)v",70,202,0.346535
2,"('assistant',)",v,"('assistant',)v",30,77,0.38961
3,"('review',)",v,"('review',)v",10,23,0.434783
4,"('pay',)",v,"('pay',)v",222,513,0.432749
...,...,...,...,...,...,...
25472,"('pandemic',)",x,"('pandemic',)x",1,2,0.5
25483,"('everyonedont',)",x,"('everyonedont',)x",1,1,1.0
25489,"('fool',)",x,"('fool',)x",1,1,1.0
25510,"('disrespectful',)",o,"('disrespectful',)o",1,1,1.0


In [208]:
#Hacer tabla prob limpia
df_uigrams_test=pd.DataFrame({"Word":[],"V":[],"X":[],"O":[]})
df_uigrams_test['Word']=valores_unigrams_unicos['index'].drop_duplicates()

In [209]:
df_uigrams_test = df_uigrams_test.fillna(0)

df_uigrams_test

,Word,V,X,O
0,"('good',)",0.0,0.0,0.0
1,"('store',)",0.0,0.0,0.0
2,"('assistant',)",0.0,0.0,0.0
3,"('review',)",0.0,0.0,0.0
4,"('pay',)",0.0,0.0,0.0
...,...,...,...,...
25470,"('corona',)",0.0,0.0,0.0
25471,"('virus',)",0.0,0.0,0.0
25483,"('everyonedont',)",0.0,0.0,0.0
25489,"('fool',)",0.0,0.0,0.0


In [210]:
colP=valores_unigrams_unicos.columns.get_loc("Probabilidad")
colRec=valores_unigrams_unicos.columns.get_loc("Recommend")
colV=df_uigrams_test.columns.get_loc("V")
colX=df_uigrams_test.columns.get_loc("X")
colO=df_uigrams_test.columns.get_loc("O")



locw=0
for word in df_uigrams_test['Word']:
    loci=0
    for index in valores_unigrams_unicos['index']:
        if word == index:
            if valores_unigrams_unicos.iloc[loci,colRec]=='v':
                df_uigrams_test.iloc[locw,colV]=valores_unigrams_unicos.iloc[loci,colP]
            if valores_unigrams_unicos.iloc[loci,colRec]=='x':
                df_uigrams_test.iloc[locw,colX]=valores_unigrams_unicos.iloc[loci,colP]
            if valores_unigrams_unicos.iloc[loci,colRec]=='o':
                df_uigrams_test.iloc[locw,colO]=valores_unigrams_unicos.iloc[loci,colP]
        loci=loci+1
    locw=locw+1 
    

In [211]:
df_uigrams_test.to_csv('ProbabilidadUniGramm_VXO.csv', index=False)


## Modelo

In [212]:
#Se abre el df test

df_glass_t_test=pd.read_csv("glassdor_test.csv")
df_glass_t_test.head(5)

#-------Se le da formato--------------


df_glass_t_test['headline'] = df_glass_t_test['headline'].fillna(",")
df_glass_t_test['pros'] = df_glass_t_test['pros'].fillna(",")
df_glass_t_test['cons'] = df_glass_t_test['cons'].fillna(",")

#Merge hedline, pros, cons
df_glass_t_test["headline+pros+cons"] = df_glass_t_test["headline"] + " " + df_glass_t_test["pros"]+ " " + df_glass_t_test["cons"]
df_glass_t_test["headline+pros+cons"]

#Make the info at column "headline+pros+cons" string
df_glass_t_test['headline+pros+cons']= df_glass_t_test['headline+pros+cons'].astype('string')

#Remove Punctuation Marks
# Apply function to the merged column
df_glass_t_test['headline+pros+cons'] = df_glass_t_test['headline+pros+cons'].apply(quitar_puntuacion)

#Remove Stop words
# Apply function to the merged column
df_glass_t_test['headline+pros+cons'] = df_glass_t_test['headline+pros+cons'].apply(quitar_stop)

#Lematize
# Aplicamos la lematización a cada fila
nlp = spacy.load("en_core_web_sm")
df_glass_t_test['headline+pros+cons_lemma'] = df_glass_t_test['headline+pros+cons'].apply(lematizar_)

#Hacer lemma string
df_glass_t_test['headline+pros+cons_lemma']=df_glass_t_test['headline+pros+cons_lemma'].astype('string')

#Remove Symbols Stop words
# Apply function to the merged column
df_glass_t_test['headline+pros+cons_cleanlemma'] = df_glass_t_test['headline+pros+cons_lemma'].apply(quitar_stop)

In [213]:
#Crea una columna para ubicar a usuario
df_glass_t_test.index.name = 'User'
#Index a columna
df_glass_t_test = df_glass_t_test.reset_index()


In [214]:
df_glass_t_test.head(5)

,User,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,...,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons,headline+pros+cons,headline+pros+cons_lemma,headline+pros+cons_cleanlemma
0,0,AFH-Wealth-Management,2015-12-11,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",2,3.0,1.0,NaN,...,4.0,x,o,r,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...,Excellent staff poor salary Friendly helpful a...,"['excellent', 'staff', 'poor', 'salary', 'frie...",excellent staff poor salary frie...
1,1,AFH-Wealth-Management,2016-04-23,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",1,2.0,1.0,NaN,...,1.0,x,o,x,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr...",client reporting admin Easy to get the job Nic...,"['client', 'report', 'admin', 'easy', 'to', 'g...",client report admin easy to g...
2,2,AFH-Wealth-Management,2017-05-21,Administrative Support,"Former Employee, more than 5 years","Birmingham, England, England",1,4.0,1.0,NaN,...,1.0,o,v,v,Administration team,Free parking . Meet some nice people in the te...,Not treated at all well after 6 yrs of being a...,Administration team Free parking Meet some ni...,"['administration', 'team', 'free', 'parking', ...",administration team free parking ...
3,3,AFH-Wealth-Management,2017-05-31,Administrative,"Current Employee, more than 3 years","Bromsgrove, England, England",5,4.0,4.0,NaN,...,5.0,o,o,o,A caring and adaptable employer,"I find AFH to be a caring employer, that is bo...",I am unable to find any Cons for AFH Wealth Ma...,A caring and adaptable employer I find AFH to ...,"['a', 'caring', 'and', 'adaptable', 'employer'...",a caring and adaptable employer ...
4,4,AFH-Wealth-Management,2017-07-02,Anonymous Employee,Former Employee,NaN,2,5.0,2.0,NaN,...,3.0,x,o,r,Administrator,Got the opportunity to work with some good peo...,Lack of progression opportunities.\r\n\r\nExpe...,Administrator Got the opportunity to work with...,"['administrator', 'get', 'the', 'opportunity',...",administrator get the opportunity ...


In [215]:
#Create new DF with unigrams,bigrams to record Y values (recommend)

colhpcl=df_glass_t_test.columns.get_loc('headline+pros+cons_cleanlemma')
colrec=df_glass_t_test.columns.get_loc('recommend')
colus=df_glass_t_test.columns.get_loc('User')


df_uigrams_test=pd.DataFrame({"count":[],"Recommend":[],"User":[]})
df_bigrams_test=pd.DataFrame({"count":[],"Recommend":[],"User":[]})


cc = 0

for x in df_glass_t_test['headline+pros+cons_cleanlemma']:

    text = df_glass_t_test.iloc[cc,colhpcl]
        
    tokens = nltk.word_tokenize(text)

    unigrams=list(ngrams(tokens, 1))
    bigrams=list(ngrams(tokens, 2))

    Ngrams=-1 #To record all

    unigram_counts = (pd.Series(unigrams).value_counts())[:Ngrams]
    bigram_counts = (pd.Series(bigrams).value_counts())[:Ngrams]

    df_new = unigram_counts.to_frame()
    df_newb = bigram_counts.to_frame()
    
    
    df_new["Recommend"]= df_glass_t_test.iloc[cc,colrec]
    df_new["User"]= df_glass_t_test.iloc[cc,colus]

    df_newb["Recommend"]= df_glass_t_test.iloc[cc,colrec]
    df_newb["User"]= df_glass_t_test.iloc[cc,colus]

    df_uigrams_test = pd.concat([df_new, df_uigrams_test])
    df_bigrams_test = pd.concat([df_newb, df_bigrams_test])

    cc = cc+ 1

In [216]:
#Index a columna
df_uigrams_test = df_uigrams_test.reset_index()
df_bigrams_test = df_bigrams_test.reset_index()

#Save the DF with unigrams test on a CSV
df_uigrams_test.to_csv('Unigrams_test.csv', index=False)
df_bigrams_test.to_csv('Bigrams_test.csv', index=False)

In [217]:
df_uigrams_test["index"]=df_uigrams_test["index"].astype(str)
df_bigrams_test["index"]=df_bigrams_test["index"].astype(str)

In [218]:
df_uigrams_test.head(3)

,index,count,Recommend,User
0,"('and',)",2.0,o,199.0
1,"('area',)",1.0,o,199.0
2,"('manager',)",1.0,o,199.0


In [219]:
df_bigrams_test.head(3)

,index,count,Recommend,User
0,"('area', 'manager')",1.0,o,199.0
1,"('manager', 'graduate')",1.0,o,199.0
2,"('be', 'very')",1.0,o,199.0


In [220]:
#Abre csvs Probabilidades

df_prob=pd.read_csv('ProbabilidadUniGramm_VXO.csv')
df_probb=pd.read_csv('ProbabilidadBiGramm_VXO.csv')

In [221]:
#Calcular Probabilidades Unigram Bigram

#Primero sacamos los usuarios unicos

usuarios_unicos=pd.DataFrame({"User":[],"ProbV":[],"ProbX":[],"ProbO":[]})
usuarios_unicosb=pd.DataFrame({"User":[],"ProbV":[],"ProbX":[],"ProbO":[]})


usuarios_unicos['User']=df_uigrams_test["User"].drop_duplicates()
usuarios_unicosb['User']=df_bigrams_test["User"].drop_duplicates()



In [222]:
#Quita Na

usuarios_unicos = usuarios_unicos.fillna(0)
usuarios_unicosb = usuarios_unicosb.fillna(0)

usuarios_unicosb.head(2)

,User,ProbV,ProbX,ProbO
0,199.0,0.0,0.0,0.0
25,198.0,0.0,0.0,0.0


In [223]:
df_prob

,Word,V,X,O
0,"('good',)",0.495495,0.325826,0.178679
1,"('store',)",0.346535,0.554455,0.099010
2,"('assistant',)",0.389610,0.467532,0.142857
3,"('review',)",0.434783,0.347826,0.217391
4,"('pay',)",0.432749,0.444444,0.122807
...,...,...,...,...
3206,"('corona',)",0.000000,1.000000,0.000000
3207,"('virus',)",0.000000,1.000000,0.000000
3208,"('everyonedont',)",0.000000,1.000000,0.000000
3209,"('fool',)",0.000000,1.000000,0.000000


In [224]:
#Calcula numeradores Probabilidades Unigram
loccu=0

colUs=df_uigrams_test.columns.get_loc('User')
colCount=df_uigrams_test.columns.get_loc('count')

colPv=usuarios_unicos.columns.get_loc('ProbV')
colPx=usuarios_unicos.columns.get_loc('ProbX')
colPo=usuarios_unicos.columns.get_loc('ProbO')

colV=df_prob.columns.get_loc('V')
colX=df_prob.columns.get_loc('X')
colO=df_prob.columns.get_loc('O')

for user in usuarios_unicos['User']:
    locct=0
    for palabra_test in df_uigrams_test["index"]:
        if user == df_uigrams_test.iloc[locct,colUs]:
            loccp=0
            for pal_prob in df_prob ["Word"]:
                if pal_prob == palabra_test:
                    if df_prob.iloc[loccp,colV] != 0:
                        if usuarios_unicos.iloc[loccu,colPv]==0:
                            usuarios_unicos.iloc[loccu,colPv]=1
                        usuarios_unicos.iloc[loccu,colPv]= usuarios_unicos.iloc[loccu,colPv]*(df_prob.iloc[loccp,colV]**df_uigrams_test.iloc[locct,colCount])
                    
                    if df_prob.iloc[loccp,colX] != 0:
                        if usuarios_unicos.iloc[loccu,colPx]==0:
                            usuarios_unicos.iloc[loccu,colPx]=1
                        usuarios_unicos.iloc[loccu,colPx]= usuarios_unicos.iloc[loccu,colPx]*(df_prob.iloc[loccp,colX]**df_uigrams_test.iloc[locct,colCount])
                    
                    if df_prob.iloc[loccp,colO] != 0:
                        if usuarios_unicos.iloc[loccu,colPo]==0:
                            usuarios_unicos.iloc[loccu,colPo]=1
                        usuarios_unicos.iloc[loccu,colPo]= usuarios_unicos.iloc[loccu,colPo]*(df_prob.iloc[loccp,colO]**df_uigrams_test.iloc[locct,colCount])
                loccp=loccp+1
        locct=locct+1
    loccu=loccu+1

In [225]:
#Calcula numeradores Probabilidades Bigram
loccu=0

colUs=df_bigrams_test.columns.get_loc('User')
colCount=df_bigrams_test.columns.get_loc('count')

colPv=usuarios_unicosb.columns.get_loc('ProbV')
colPx=usuarios_unicosb.columns.get_loc('ProbX')
colPo=usuarios_unicosb.columns.get_loc('ProbO')

colV=df_probb.columns.get_loc('V')
colX=df_probb.columns.get_loc('X')
colO=df_probb.columns.get_loc('O')

for user in usuarios_unicosb['User']:
    locct=0
    for palabra_test in df_bigrams_test["index"]:
        if user == df_bigrams_test.iloc[locct,colUs]:
            loccp=0
            for pal_probb in df_probb ["Word"]:
                if pal_probb == palabra_test:
                    if df_probb.iloc[loccp,colV] != 0:
                        if usuarios_unicosb.iloc[loccu,colPv]==0:
                            usuarios_unicosb.iloc[loccu,colPv]=1
                        usuarios_unicosb.iloc[loccu,colPv]= usuarios_unicosb.iloc[loccu,colPv]*(df_probb.iloc[loccp,colV]**df_bigrams_test.iloc[locct,colCount])
                    
                    if df_probb.iloc[loccp,colX] != 0:
                        if usuarios_unicosb.iloc[loccu,colPx]==0:
                            usuarios_unicosb.iloc[loccu,colPx]=1
                        usuarios_unicosb.iloc[loccu,colPx]= usuarios_unicosb.iloc[loccu,colPx]*(df_probb.iloc[loccp,colX]**df_bigrams_test.iloc[locct,colCount])
                    
                    if df_probb.iloc[loccp,colO] != 0:
                        if usuarios_unicosb.iloc[loccu,colPo]==0:
                            usuarios_unicosb.iloc[loccu,colPo]=1
                        usuarios_unicosb.iloc[loccu,colPo]= usuarios_unicosb.iloc[loccu,colPo]*(df_probb.iloc[loccp,colO]**df_bigrams_test.iloc[locct,colCount])
                loccp=loccp+1
        locct=locct+1
    loccu=loccu+1

In [228]:
print("Cálculo de numeradores por Usuario Bigrama:")
usuarios_unicosb

Cálculo de numeradores por Usuario Bigrama:


,User,ProbV,ProbX,ProbO
0,199.0,4.000430e-03,4.388432e-02,5.059833e-03
25,198.0,1.224490e-02,3.214286e-01,9.183673e-03
41,197.0,1.315590e-05,6.896775e-03,1.831460e-05
62,196.0,6.610270e-02,6.138107e-02,9.836711e-05
71,195.0,3.525641e-03,1.182692e-01,2.500000e-01
...,...,...,...,...
9734,4.0,4.168671e-04,1.106301e-02,3.817464e-06
9762,3.0,2.028038e-07,2.054913e-08,3.468119e-12
9909,2.0,2.772630e-08,1.550710e-05,2.420076e-08
10055,1.0,3.401361e-03,9.353741e-02,1.000000e+00


In [258]:
#Normalizacion para encontrar las probabilidades reales Unigrama

#Se saca el total de las probabilidades por usuario

usuarios_unicos["SumProb"]=None
#Cálculo probabilidad normalizada
usuarios_unicos["ProbNomV%"]=None
usuarios_unicos["ProbNomX%"]=None
usuarios_unicos["ProbNomO%"]=None


colPv=usuarios_unicos.columns.get_loc('ProbV')
colPx=usuarios_unicos.columns.get_loc('ProbX')
colPo=usuarios_unicos.columns.get_loc('ProbO')

colPvp=usuarios_unicos.columns.get_loc("ProbNomV%")
colPxp=usuarios_unicos.columns.get_loc("ProbNomX%")
colPop=usuarios_unicos.columns.get_loc("ProbNomO%")

colSum=usuarios_unicos.columns.get_loc('SumProb')

locs=0

for usuario in usuarios_unicos["SumProb"]:
    usuarios_unicos.iloc[locs,colSum]=usuarios_unicos.iloc[locs,colPv]+usuarios_unicos.iloc[locs,colPx]+usuarios_unicos.iloc[locs,colPo]
    locs=locs+1

locn=0

for usuario in usuarios_unicos["ProbNomV%"]:
    if usuarios_unicos.iloc[locn,colSum] !=0:
        usuarios_unicos.iloc[locn,colPvp]=round((usuarios_unicos.iloc[locn,colPv]/usuarios_unicos.iloc[locn,colSum])*100,2)
        usuarios_unicos.iloc[locn,colPxp]=round((usuarios_unicos.iloc[locn,colPx]/usuarios_unicos.iloc[locn,colSum])*100,2)
        usuarios_unicos.iloc[locn,colPop]=round((usuarios_unicos.iloc[locn,colPo]/usuarios_unicos.iloc[locn,colSum])*100,2)
    else:
        usuarios_unicos.iloc[locn,colPvp]=0
        usuarios_unicos.iloc[locn,colPxp]=0
        usuarios_unicos.iloc[locn,colPop]=0
    locn=locn+1

In [257]:
#Normalizacion para encontrar las probabilidades reales Bigrama

#Se saca el total de las probabilidades por usuario

usuarios_unicosb["SumProb"]=None

#Cálculo probabilidad normalizada
usuarios_unicosb["ProbNomV%"]=None
usuarios_unicosb["ProbNomX%"]=None
usuarios_unicosb["ProbNomO%"]=None

colPv=usuarios_unicosb.columns.get_loc('ProbV')
colPx=usuarios_unicosb.columns.get_loc('ProbX')
colPo=usuarios_unicosb.columns.get_loc('ProbO')

colPvp=usuarios_unicosb.columns.get_loc("ProbNomV%")
colPxp=usuarios_unicosb.columns.get_loc("ProbNomX%")
colPop=usuarios_unicosb.columns.get_loc("ProbNomO%")

colSum=usuarios_unicosb.columns.get_loc('SumProb')

locs=0

for usuario in usuarios_unicosb["SumProb"]:
    usuarios_unicosb.iloc[locs,colSum]=usuarios_unicosb.iloc[locs,colPv]+usuarios_unicosb.iloc[locs,colPx]+usuarios_unicosb.iloc[locs,colPo]
    locs=locs+1

locn=0

for usuario in usuarios_unicosb["ProbNomV%"]:
    if usuarios_unicosb.iloc[locn,colSum] !=0:
        usuarios_unicosb.iloc[locn,colPvp]=round((usuarios_unicosb.iloc[locn,colPv]/usuarios_unicosb.iloc[locn,colSum])*100,2)
        usuarios_unicosb.iloc[locn,colPxp]=round((usuarios_unicosb.iloc[locn,colPx]/usuarios_unicosb.iloc[locn,colSum])*100,2)
        usuarios_unicosb.iloc[locn,colPop]=round((usuarios_unicosb.iloc[locn,colPo]/usuarios_unicosb.iloc[locn,colSum])*100,2)
    else:
        usuarios_unicosb.iloc[locn,colPvp]=0
        usuarios_unicosb.iloc[locn,colPxp]=0
        usuarios_unicosb.iloc[locn,colPop]=0
    locn=locn+1

In [235]:
#Comparar resultados obtenidos vs train Unigram

usuarios_unicos['Answer Predicted']= ""

colPvp=usuarios_unicos.columns.get_loc("ProbNomV%")
colPxp=usuarios_unicos.columns.get_loc("ProbNomX%")
colPop=usuarios_unicos.columns.get_loc("ProbNomO%")

colAnp=usuarios_unicos.columns.get_loc('Answer Predicted')

locap=0
for user in usuarios_unicos['User']:
    xVal=usuarios_unicos.iloc[locap,colPxp]
    vVal=usuarios_unicos.iloc[locap,colPvp]
    oVal=usuarios_unicos.iloc[locap,colPop]

    if xVal > vVal and xVal > oVal:
        usuarios_unicos.iloc[locap,colAnp]='x'
    if vVal > xVal and vVal > oVal:
        usuarios_unicos.iloc[locap,colAnp]='v'
    if oVal > vVal and oVal > xVal:
        usuarios_unicos.iloc[locap,colAnp]='o'
    locap=locap+1


In [240]:
#Comparar resultados obtenidos vs train Bigram

usuarios_unicosb['Answer Predicted']= ""

colPvp=usuarios_unicosb.columns.get_loc("ProbNomV%")
colPxp=usuarios_unicosb.columns.get_loc("ProbNomX%")
colPop=usuarios_unicosb.columns.get_loc("ProbNomO%")

colAnp=usuarios_unicosb.columns.get_loc('Answer Predicted')

locap=0
for user in usuarios_unicosb['User']:
    xVal=usuarios_unicosb.iloc[locap,colPxp]
    vVal=usuarios_unicosb.iloc[locap,colPvp]
    oVal=usuarios_unicos.iloc[locap,colPop]

    if xVal > vVal and xVal > oVal:
        usuarios_unicosb.iloc[locap,colAnp]='x'
    if vVal > xVal and vVal > oVal:
        usuarios_unicosb.iloc[locap,colAnp]='v'
    if oVal > vVal and oVal > xVal:
        usuarios_unicosb.iloc[locap,colAnp]='o'
    locap=locap+1


In [241]:
usuarios_unicosb.head(2)

,User,ProbV,ProbX,ProbO,SumProb,ProbNomV%,ProbNomX%,ProbNomO%,Answer Predicted
0,199.0,0.004000,0.043884,0.005060,0.052945,7.56,82.89,9.56,x
25,198.0,0.012245,0.321429,0.009184,0.342857,3.57,93.75,2.68,x


In [242]:
usuarios_unicos.to_csv('Results_test_Unigram.csv', index=False)
usuarios_unicosb.to_csv('Results_test_Bigram.csv', index=False)

In [250]:
#Concatenar respuestas presichas y reales Unigrama
usuarios_unicos['Real Answer']=""

colRA=usuarios_unicos.columns.get_loc("Real Answer")

colRm=df_uigrams_test.columns.get_loc("Recommend")


locus=0
for user in usuarios_unicos["User"]:
    locni=0
    for useruni in df_uigrams_test["User"]:
        if user == useruni:
            usuarios_unicos.iloc[locus,colRA]=df_uigrams_test.iloc[locni,colRm]
        locni=locni+1
    locus=locus+1

In [244]:
#Concatenar respuestas presichas y reales Bigrama


usuarios_unicosb['Real Answer']=""

colRA=usuarios_unicosb.columns.get_loc("Real Answer")

colRm=df_bigrams_test.columns.get_loc("Recommend")


locus=0
for user in usuarios_unicosb["User"]:
    locni=0
    for useruni in df_bigrams_test["User"]:
        if user == useruni:
            usuarios_unicosb.iloc[locus,colRA]=df_bigrams_test.iloc[locni,colRm]
        locni=locni+1
    locus=locus+1

## Accuracy

In [259]:
y_pred=usuarios_unicos["Answer Predicted"]
y_test=usuarios_unicos["Real Answer"]

y_predb=usuarios_unicosb["Answer Predicted"]
y_testb=usuarios_unicosb["Real Answer"]

tP=0
sUM=0
cc=0

for x in y_pred:
    sUM=sUM+1
    if x == y_test.iloc[cc]:
        tP = tP + 1
    cc=cc+1

tPb=0
sUMb=0
ccb=0

for x in y_predb:
    sUMb=sUMb+1
    if x == y_testb.iloc[ccb]:
        tPb = tPb + 1
    ccb=ccb+1

print(f'Accuracy Unigrama: {tP/sUM}')
print(f'Accuracy Bigrama: {tPb/sUMb}')



Accuracy Unigrama: 0.505
Accuracy Bigrama: 0.605
